# 基于MindNLP的Roberta模型Prompt Tuning

## 环境安装

In [1]:
#安装mindnlp 0.4.0套件
!pip install mindnlp==0.4.0
!pip uninstall soundfile -y
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.3.1/MindSpore/unified/aarch64/mindspore-2.3.1-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

In [1]:
%env HF_ENDPOINT=https://hf-mirror.com

env: HF_ENDPOINT=https://hf-mirror.com


## 模型与数据集加载

本案例对roberta-large模型基于GLUE基准数据集进行prompt tuning。

In [2]:
import argparse
import os

import mindspore
from mindnlp.core.optim import AdamW
from tqdm import tqdm
import evaluate
from mindnlp.dataset import load_dataset
from mindnlp.engine import set_seed
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup
from mindnlp.peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptTuningConfig,
)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.277 seconds.
Prefix dict has been built successfully.


In [3]:
batch_size = 32
model_name_or_path = "AI-ModelScope/roberta-large"
task = "mrpc"
peft_type = PeftType.PROMPT_TUNING
# num_epochs = 20
num_epochs = 5

prompt tuning配置，任务类型选为"SEQ_CLS", 即序列分类。

In [4]:
# peft config
peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=10)
# learning rate
lr = 1e-3

加载tokenizer。如模型为GPT、OPT或BLOOM类模型，从序列左侧添加padding，其他情况下从序列右侧添加padding。

In [5]:
# load tokenizer
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side, mirror="modelscope")
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [6]:
datasets = load_dataset("glue", task)
print(next(datasets['train'].create_dict_iterator()))

{'sentence1': Tensor(shape=[], dtype=String, value= 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'), 'sentence2': Tensor(shape=[], dtype=String, value= 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'), 'label': Tensor(shape=[], dtype=Int64, value= 1), 'idx': Tensor(shape=[], dtype=Int64, value= 0)}


In [7]:
from mindnlp.dataset import BaseMapFunction

class MapFunc(BaseMapFunction):
    def __call__(self, sentence1, sentence2, label, idx):
        outputs = tokenizer(sentence1, sentence2, truncation=True, max_length=None)
        return outputs['input_ids'], outputs['attention_mask'], label


def get_dataset(dataset, tokenizer):
    input_colums=['sentence1', 'sentence2', 'label', 'idx']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})
    return dataset

train_dataset = get_dataset(datasets['train'], tokenizer)
eval_dataset = get_dataset(datasets['validation'], tokenizer)

In [8]:
print(next(train_dataset.create_dict_iterator()))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': Tensor(shape=[32, 70], dtype=Int64, value=
[[    0, 10127,  1001 ...     1,     1,     1],
 [    0,   975, 26802 ...     1,     1,     1],
 [    0,  1213,    56 ...     1,     1,     1],
 ...
 [    0,   133,  1154 ...     1,     1,     1],
 [    0, 12667,  8423 ...     1,     1,     1],
 [    0, 32478,  1033 ...     1,     1,     1]]), 'attention_mask': Tensor(shape=[32, 70], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[32], dtype=Int64, value= [1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 
 1, 1, 0, 0, 1, 1, 1, 0])}


In [9]:
metric = evaluate.load("glue", task)

加载模型并打印微调参数量，可以看到仅有不到0.6%的参数参与了微调。

如出现如下告警请忽略，并不影响模型的微调。

```text
The following parameters in checkpoint files are not loaded:
['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.embeddings.position_ids']
The following parameters in models are missing parameter:
['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
```

In [10]:
# load model
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True, mirror="modelscope")
model = get_peft_model(model, peft_config)
# print number of trainable parameters
model.print_trainable_parameters()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at AI-ModelScope/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,061,890 || all params: 356,423,684 || trainable%: 0.2979291353713745


## 模型微调（prompt tuning）

指定优化器和学习率调整策略

In [11]:
optimizer = AdamW(params=model.trainable_params(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataset) * num_epochs),
    num_training_steps=(len(train_dataset) * num_epochs),
)

打印参与微调的模型参数

In [12]:
# print name of trainable parameters
model.trainable_params()

(Tensor(shape=[1024, 1024], dtype=Float32, value=
 [[ 9.42201912e-03,  3.68431699e-03, -3.65935750e-02 ...  4.10799198e-02, -6.41081808e-03,  8.61441996e-03],
  [ 1.17747569e-02,  9.00278613e-03,  1.73328817e-02 ...  2.88665313e-02, -2.76252581e-03, -5.72031247e-04],
  [-4.60726238e-04,  1.10795023e-02, -5.18864894e-04 ... -1.99680869e-02,  7.68191880e-03,  4.72938782e-03],
  ...
  [ 9.52273549e-04,  9.51045938e-03,  2.76891738e-02 ...  1.45859383e-02,  2.44282447e-02,  2.00108700e-02],
  [ 7.29785347e-03, -1.26254419e-02,  8.73612612e-03 ...  1.38808256e-02, -1.82791520e-02,  4.01444733e-02],
  [ 2.23141722e-03,  1.06983585e-02, -7.14888738e-04 ...  1.49706863e-02,  2.54806578e-02, -8.68784171e-03]]),
 Tensor(shape=[1024], dtype=Float32, value= [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 Tensor(shape=[2, 1024], dtype=Float32, value=
 [[ 9.76719335e-03, -1.01758288e-02,  1.73892733e-03 ...  1.86475646e-02, -1.13016569e-04

按照如下步骤定义训练逻辑：

1. 构建正向计算函数
2. 函数变换，获取微分函数
3. 定义训练一个step的逻辑
4. 遍历训练数据集进行模型训练，同时每一个epoch后，遍历验证数据集获取当前的评价指标（accuracy、f1 score）

In [14]:
# define forward function
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

# Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())

# Define function of one-step training
def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer.step(grads)
    return loss

# Start training
for epoch in range(num_epochs):
    model.set_train()
    train_total_size = train_dataset.get_dataset_size()
    # Iterate through the dataset
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        lr_scheduler.step()

    # Evaluate while training
    model.set_train(False)
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        outputs = model(**batch)
        predictions = outputs.logits.argmax(axis=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )
    
    # Calculate accuracy and f1 score
    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

  1%|          | 1/115 [00:02<03:57,  2.08s/it]

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:03<00:00,  4.06it/s]


epoch 0: {'accuracy': 0.7058823529411765, 'f1': 0.8192771084337349}


100%|██████████| 13/13 [00:01<00:00,  7.01it/s]


...

epoch 2: {'accuracy': 0.7156862745098039, 'f1': 0.8176100628930817}


100%|██████████| 13/13 [00:01<00:00,  7.11it/s]


epoch 3: {'accuracy': 0.7083333333333334, 'f1': 0.8205128205128206}


100%|██████████| 13/13 [00:01<00:00,  6.99it/s]


epoch 4: {'accuracy': 0.7132352941176471, 'f1': 0.8229954614220877}
